In [1]:
!pip install /content/siameseFastTraining-0.1.3-py3-none-any.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./siameseFastTraining-0.1.3-py3-none-any.whl
siameseFastTraining is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [2]:
import os
import random
from torchvision import transforms
from PIL import Image
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, roc_curve
from efficientnet_pytorch import EfficientNet
from collections import Counter

from siameseFastTraining.SiameseNetworks import TripletDataset, EfficientNetFeatureExtractor, SiameseNetwork, Trainer
from siameseFastTraining.LossFunctions import TripletLoss
from siameseFastTraining.utils import count_trainable_parameters

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"

### Bit of testing....

In [4]:
train_dataset = TripletDataset(root_dir="/content/drive/MyDrive/ULTRADATA/Motos_reencuadradas/Train", transform=None)

In [5]:
train_dataset.get_subject_image_path(0)

('14',
 '/content/drive/MyDrive/ULTRADATA/Motos_reencuadradas/Train/14/032423001682.JPG')

In [6]:
train_dataset.get_positive_image('14', '/content/drive/MyDrive/ULTRADATA/Motos_reencuadradas/Train/14/032423001682.JPG')

'/content/drive/MyDrive/ULTRADATA/Motos_reencuadradas/Train/14/032423001681.JPG'

# Create the Siamese Datasets

## Dataset

In [15]:
img_size = 224

train_imgs_path = "/content/drive/MyDrive/ULTRADATA/Motos_reencuadradas/Train"
test_imgs_path  = "/content/drive/MyDrive/ULTRADATA/Motos_reencuadradas/Test"

transform_train = transforms.Compose([
  transforms.Resize((img_size, img_size)),
  transforms.RandomHorizontalFlip(),
  transforms.RandomRotation(15),
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  # , transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)
])

transform_valid = transforms.Compose([
    transforms.Resize((img_size, img_size)), # transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


siamese_train_dataset = TripletDataset(root_dir=train_imgs_path, transform=transform_train)
siamese_test_dataset = TripletDataset(root_dir=test_imgs_path, transform=transform_valid)

## DataLoader

In [16]:
batch_size = 32
shuffle = True
num_workers = 2

siamese_train_dataloader = DataLoader(siamese_train_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
siamese_val_dataloader   = DataLoader(siamese_test_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

# Creating Neural Network

## Feature Extractor

In [17]:
feature_extractor = EfficientNetFeatureExtractor(model_name='efficientnet-b0', pretrained=True)
feature_extractor = feature_extractor.to(device)

Loaded pretrained weights for efficientnet-b0


## Siamese Network

In [18]:
siamese_network = SiameseNetwork(
    feature_extractor=feature_extractor, 
    model_name="EfficientNet", 
    init_method=None, 
    batch_norm=None, 
    learning_rate=1e-4, 
    epochs=20, 
    dataset="Motos_recortes", 
    loss_function="triplet_loss", 
    accuracy_threshold=0.5, 
    img_size=img_size
)

n_params = count_trainable_parameters(siamese_network)
print(f"Model's trainable parameters: {n_params}")

Model's trainable parameters: 328448


## Loss Function

In [19]:
loss_function = TripletLoss()

optimizer = optim.Adam(filter(lambda p: p.requires_grad, siamese_network.parameters()), lr=siamese_network.learning_rate)

# Training the model

In [20]:
trainer = Trainer(
    siamese_network, 
    model_name = siamese_network.name,
    init_method = "None",
    batch_norm = siamese_network.batch_norm,
    learning_rate = siamese_network.learning_rate,
    epochs = 50, 
    dataset = siamese_network.dataset,
    img_size = siamese_network.img_size,
    optimizer = optimizer,
    loss_function = loss_function
)

In [21]:
trainer.train(siamese_train_dataloader, siamese_val_dataloader)

Validation: 100%|██████████| 2/2 [00:15<00:00,  7.70s/it]

Epoch: 0, Train Loss: 21.0841, Train Accuracy: 0.6781, Val Loss: 7.9301, Val Accuracy: 0.8117



Validation: 100%|██████████| 2/2 [00:05<00:00,  3.00s/it]

Epoch: 1, Train Loss: 18.3530, Train Accuracy: 0.7094, Val Loss: 13.3267, Val Accuracy: 0.8536



Validation: 100%|██████████| 2/2 [00:04<00:00,  2.29s/it]


...

Epoch: 17, Train Loss: 10.7436, Train Accuracy: 0.8250, Val Loss: 3.2060, Val Accuracy: 0.8692


Validation: 100%|██████████| 2/2 [00:04<00:00,  2.10s/it]


Epoch: 18, Train Loss: 10.6508, Train Accuracy: 0.8187, Val Loss: 12.2296, Val Accuracy: 0.8166


Validation: 100%|██████████| 2/2 [00:03<00:00,  1.98s/it]


Epoch: 19, Train Loss: 9.8484, Train Accuracy: 0.8344, Val Loss: 12.0197, Val Accuracy: 0.7952


## Save the model results

In [23]:
epochs = siamese_network.epochs
dataset = siamese_network.dataset
loss_fn = siamese_network.loss_function
model_name = siamese_network.name
lr = siamese_network.learning_rate

model_path = f"""/content/drive/MyDrive/ULTRADATA/trained_models/name-{model_name}_lr-{lr}_epochs-{epochs}_dataset-{dataset}_loss-fn-{loss_fn}.pth"""

print(model_path)

/content/drive/MyDrive/ULTRADATA/trained_models/name-EfficientNet_lr-0.0001_epochs-20_dataset-Motos_recortes_loss-fn-triplet_loss.pth


In [24]:
torch.save(siamese_network.state_dict(), model_path)